<a href="https://colab.research.google.com/github/Davidjr2889/Davidjr2889/blob/main/ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd


from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [4]:
data = pd.read_csv('MELBOURNE_HOUSE_PRICES_LESS.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63023 entries, 0 to 63022
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         63023 non-null  object 
 1   Address        63023 non-null  object 
 2   Rooms          63023 non-null  int64  
 3   Type           63023 non-null  object 
 4   Price          48433 non-null  float64
 5   Method         63023 non-null  object 
 6   SellerG        63023 non-null  object 
 7   Date           63023 non-null  object 
 8   Postcode       63023 non-null  int64  
 9   Regionname     63023 non-null  object 
 10  Propertycount  63023 non-null  int64  
 11  Distance       63023 non-null  float64
 12  CouncilArea    63023 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 6.3+ MB


In [6]:
#preprocessamento

In [7]:
def onehot_encode(df, column_dict):
  df = df.copy()

  for column, prefix in column_dict.items():
    dummies = pd.get_dummies(df[column], prefix= prefix)
    df = pd.concat([df, dummies], axis= 1)
    df = df.drop(column, axis=1)
  return df

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

Criar um espaço para alocar o endereço o qual será categorizado.

In [49]:
def get_sequences(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)

    vocab_length = len(tokenizer.word_index) + 1
    print('Vocab length :', vocab_length)
   

    sequences = tokenizer.texts_to_sequences(texts)

   

    max_seq_length = np.max([len(sequence) for sequence in sequences])
    print('Maximo de sequencias:', max_seq_length)

    sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')
    
    return sequences

In [50]:
get_sequences(X['Address'])

Vocab length : 10774
Maximo de sequencias: 5


array([[  66, 2804,    1,    0,    0],
       [2408,  418,    1,    0,    0],
       [6811,  313,    1,    0,    0],
       ...,
       [   7, 3345,   26,    0,    0],
       [  15,  779,    6,    0,    0],
       [  10, 6711,   36,    0,    0]], dtype=int32)

In [52]:
pd.get_dummies(data['Method'])

,PI,PN,S,SA,SN,SP,SS,VB,W
0,0,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
63018,0,0,1,0,0,0,0,0,0
63019,0,0,1,0,0,0,0,0,0
63020,0,0,1,0,0,0,0,0,0
63021,1,0,0,0,0,0,0,0,0


In [56]:

def preprocess_inputs(df):
  df = df.copy()
  #tirando linhas com valores vazios

  df = df.dropna(axis = 0 ).reset_index(drop=True)

  #one-hotencoder

  column_dict = {
      'Suburb': 'SU',
      'Type': 'TY',  
      'Method': 'ME',   
      'SellerG': 'SE', 
      'Postcode': 'PO',   
      'Regionname': 'RE',
      'CouncilArea': 'CO',
  }
 
  df = onehot_encode(df, column_dict)

  # Reemplazando Data por --- Ano, Mes, e dia com metodo lambda
  df['Date'] = pd.to_datetime(df['Date'])

  df['Year'] = df['Date'].apply(lambda x: x.year)
  df['Month'] = df['Date'].apply(lambda x: x.month)
  df['Day'] = df['Date'].apply(lambda x: x.day)

  df = df.drop('Date', axis=1)

  #obter sequencias de endereços

  addr_sequences = get_sequences(df['Address'])
  df = df.drop('Address', axis = 1)

  # Decl
  y = df['Price'].copy()
  X = df.drop('Price', axis = 1).copy()

  return X, addr_sequences, y


In [57]:
X, addr_sequences, y = preprocess_inputs(data)

Vocab length : 10774
Maximo de sequencias: 5


In [18]:
X


,Rooms,Propertycount,Distance,SU_Abbotsford,SU_Aberfeldie,SU_Airport West,SU_Albanvale,SU_Albert Park,SU_Albion,SU_Alphington,SU_Altona,SU_Altona Meadows,SU_Altona North,SU_Ardeer,SU_Armadale,SU_Ascot Vale,SU_Ashburton,SU_Ashwood,SU_Aspendale,SU_Aspendale Gardens,SU_Attwood,SU_Avondale Heights,SU_Bacchus Marsh,SU_Balaclava,SU_Balwyn,SU_Balwyn North,SU_Bayswater,SU_Bayswater North,SU_Beaconsfield,SU_Beaconsfield Upper,SU_Beaumaris,SU_Belgrave,SU_Belgrave Heights,SU_Belgrave South,SU_Bellfield,SU_Bentleigh,SU_Bentleigh East,SU_Berwick,SU_Black Rock,SU_Blackburn,...,RE_Southern Metropolitan,RE_Western Metropolitan,RE_Western Victoria,CO_Banyule City Council,CO_Bayside City Council,CO_Boroondara City Council,CO_Brimbank City Council,CO_Cardinia Shire Council,CO_Casey City Council,CO_Darebin City Council,CO_Frankston City Council,CO_Glen Eira City Council,CO_Greater Dandenong City Council,CO_Hobsons Bay City Council,CO_Hume City Council,CO_Kingston City Council,CO_Knox City Council,CO_Macedon Ranges Shire Council,CO_Manningham City Council,CO_Maribyrnong City Council,CO_Maroondah City Council,CO_Melbourne City Council,CO_Melton City Council,CO_Mitchell Shire Council,CO_Monash City Council,CO_Moonee Valley City Council,CO_Moorabool Shire Council,CO_Moreland City Council,CO_Murrindindi Shire Council,CO_Nillumbik Shire Council,CO_Port Phillip City Council,CO_Stonnington City Council,CO_Whitehorse City Council,CO_Whittlesea City Council,CO_Wyndham City Council,CO_Yarra City Council,CO_Yarra Ranges Shire Council,Year,Month,Day
0,3,4019,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2017,1,4
1,3,4019,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2017,1,4
2,3,4019,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2017,1,4
3,3,1543,7.5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2017,1,4
4,2,3464,10.4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2017,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48428,2,17055,38.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,31
48429,3,14577,8.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,31
48430,3,5833,20.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,31
48431,3,5833,20.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,31


In [21]:
addr_sequences

array([[  66, 2804,    1,    0,    0],
       [2408,  418,    1,    0,    0],
       [6811,  313,    1,    0,    0],
       ...,
       [   7, 3345,   26,    0,    0],
       [  15,  779,    6,    0,    0],
       [  10, 6711,   36,    0,    0]], dtype=int32)

In [120]:
y

0        1490000.0
1        1220000.0
2        1420000.0
3        1515000.0
4         670000.0
           ...    
48428     347700.0
48429     808000.0
48430     566000.0
48431     500000.0
48432     545000.0
Name: Price, Length: 48433, dtype: float64